In [7]:
import numpy as np
import sys
from pyproj import Transformer, CRS

'''
@functions: rotation matrices
@param: angle given in degree
@output: corresponding rotation matrices
'''
def roll(angle):
    angle = np.radians(angle)
    return np.array([[np.cos(angle),-np.sin(angle),0],[np.sin(angle),np.cos(angle),0],[0,0,1]])

def yaw(angle):
    angle = np.radians(angle)
    return np.array([[1,0,0],[0,np.cos(angle),-np.sin(angle)],[0,np.sin(angle),np.cos(angle)]])

def pitch(angle):
    angle = np.radians(angle)
    return np.array([[np.cos(angle),0,np.sin(angle)],[0,1,0],[-np.sin(angle),0,np.cos(angle)]])

'''
@function: convert the GPS location into the flat rectangular grid 
           by using the projection from WGS86 to LV03 -> ONLY SUITABLE FOR SWITZERLAND!
@param: gps - [lat, lng, alt]
        lat - latitude
        lng - longitude
        alt - altitude
@output: the projected location of the object in LV03 Eucledian coordinate
'''
def gps2world(gps):
    lat = gps[0]
    lng = gps[1]
    alt = gps[2]
    # 4326 is the identifier for WGS 86
    crs_4326 = CRS.from_epsg(4326) 
    # 21781 is the identifier for LV03/CH1903
    crs_21781 = CRS.from_epsg(21781)
    transformer_4326_2_21781 = Transformer.from_crs(crs_4326,crs_21781)
    pos = transformer_4326_2_21781.transform(lat,lng)
    return np.concatenate([np.array(pos),alt],axis=None)

'''
@function: convert the into the flat rectangular grid GPS location
           by using the projection from LV03 to WGS86 -> ONLY SUITABLE FOR SWITZERLAND!
@param: world - [x,y,z]
        x - x-axis
        y - y-axis
        z - z-axis
@output: the projected location of the object in gps coordinate
'''
def world2gps(world):
    x = world[0]
    y = world[1]
    z = world[2]
    # 4326 is the identifier for WGS 86
    crs_4326 = CRS.from_epsg(4326) 
    # 21781 is the identifier for LV03/CH1903
    crs_21781 = CRS.from_epsg(21781)
    transformer_21781_2_4326 = Transformer.from_crs(crs_21781,crs_4326)
    gps = transformer_21781_2_4326.transform(x,y)
    return np.concatenate([np.array(gps),z],axis=None)

'''
@function: convert the world coordinate into the body coordinate
@param: R - the rotation matrix from world to body
        t - the translation vector world to body
        x_in - the input vector in the inhomogeneous coordinate
        n - the dimension of the input vector
        init - the initial point in
@output: the object in the body coordinate
@TODO: ADD R AND T AS DEFAULT VALUE
'''
def world2body(x_in, init):
    R = init.yaw(init.angle)
    t = init.world
    return np.dot(get_transformation_matrix(R,t),x_in)

def body2world(x_in, init):
    R = init.yaw(init.angle)
    t = init.world
    return np.dot(np.linalg.inv(get_transformation_matrix(R,t)),x_in)


ModuleNotFoundError: No module named 'pyproj'

In [6]:
!pip3 install pyproj

  Using cached https://files.pythonhosted.org/packages/2c/12/7a8cca32506747c05ffd5c6ba556cf8435754af0939906cbcc7fa5802ea3/pyproj-3.0.1.tar.gz
    Complete output from command python setup.py egg_info:
    ERROR: Minimum supported PROJ version is 7.2.0, installed version is 4.9.3. For more information see: https://pyproj4.github.io/pyproj/stable/installation.html
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-build-_7ypcpz4/pyproj/
